In [4]:
# Import Libraries
from binance.client import Client as bnb_client
from datetime import datetime
import pandas as pd
import time
import os

In [5]:
# Step 1: Fetch Raw Data Function
client = bnb_client(tld='us')

def fetch_raw_data(universe, freq='4h', start='2020-01-01', end='2025-06-01'):
    """
    Fetch full raw OHLCV data from Binance for an inputed list of coins (universe).
    Returns a dictionary of DataFrames indexed by open_time.
    """
    all_data = {}
    
    # Column names returned by Binance
    columns = [
        'open_time', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_volume', 'num_trades',
        'taker_base_volume', 'taker_quote_volume', 'ignore'
    ]
    
    for symbol in universe:
        print(f"Fetching {symbol}...")
        try:
            raw = client.get_historical_klines(symbol, freq, start, end)
            df = pd.DataFrame(raw, columns=columns)
            
            # Convert open_time to datetime index
            df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
            df.set_index('open_time', inplace=True)
            
            # Convert numeric fields
            for col in columns[1:11]:  # skip 'open_time' and 'ignore'
                df[col] = pd.to_numeric(df[col], errors='coerce')
            
            all_data[symbol] = df
            time.sleep(1)
        
        except Exception as e:
            print(f"Error fetching {symbol}: {e}")
    
    return all_data

In [6]:
# Set universe and fetch raw data
univ = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BNBUSDT', 'SOLUSDT']
data = fetch_raw_data(univ)

# Trim XRP
xrp_cutoff = pd.Timestamp('2023-07-14 12:00:00')
if 'XRPUSDT' in data:
    data['XRPUSDT'] = data['XRPUSDT'].loc[data['XRPUSDT'].index >= xrp_cutoff]

"""for symbol, df in data.items():
    df.to_csv(f"raw_{symbol}_4h.csv")"""

Fetching BTCUSDT...
Fetching ETHUSDT...
Fetching XRPUSDT...
Fetching BNBUSDT...
Fetching SOLUSDT...


'for symbol, df in data.items():\n    df.to_csv(f"raw_{symbol}_4h.csv")'

In [11]:
# Step 2: Features Functions
def compute_returns(data_dict):
    returns_dict = {}
    for symbol, df in data_dict.items():
        df = df.sort_index()  # Ensure time order
        ret_df = df[['close']].pct_change().rename(columns={'close': 'return'})
        returns_dict[symbol] = ret_df
    return returns_dict

"""def compute_volatility(data_dict, short_win=7, mid_win=42, long_win=126):
    volatility = {}

    for symbol, df in data_dict.items():
        vol_df = pd.DataFrame(index=df.index)
        vol_df['vol_short'] = df['return'].rolling(window=short_win).std()
        vol_df['vol_medium'] = df['return'].rolling(window=mid_win).std()
        vol_df['vol_long'] = df['return'].rolling(window=long_win).std()
        volatility[symbol] = vol_df

    return volatility"""

def compute_volatility(data_dict):
    volatility = {}

    for symbol, df in data_dict.items():
        vol_df = pd.DataFrame(index=df.index)
        vol_df['volatility'] = df['return'].abs()  # one-point proxy for volatility
        volatility[symbol] = vol_df

    return volatility

def extract_volume(data_dict):
    volume_dict = {}
    for symbol, df in data_dict.items():
        volume_df = df[['volume']].copy()
        volume_dict[symbol] = volume_df
    return volume_dict

def compute_rsi(data_dict, rsi_window=14):
    rsi_dict = {}

    for symbol, df in data_dict.items():
        delta = df['close'].diff()

        gain = delta.where(delta > 0, 0.0)
        loss = -delta.where(delta < 0, 0.0)

        avg_gain = gain.rolling(window=rsi_window).mean()
        avg_loss = loss.rolling(window=rsi_window).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))

        rsi_df = pd.DataFrame(index=df.index)
        rsi_df['rsi'] = rsi

        rsi_dict[symbol] = rsi_df

    return rsi_dict

def compute_sma(data_dict, windows=[10, 50, 200]):
    sma_dict = {}
    for symbol, df in data_dict.items():
        sma_df = pd.DataFrame(index=df.index)
        for window in windows:
            sma_df[f'sma_{window}'] = df['close'].rolling(window=window).mean()
        sma_dict[symbol] = sma_df
    return sma_dict


def compute_ema(data_dict, windows=[12, 26, 50]):
    ema_dict = {}
    for symbol, df in data_dict.items():
        ema_df = pd.DataFrame(index=df.index)
        for window in windows:
            ema_df[f'ema_{window}'] = df['close'].ewm(span=window, adjust=False).mean()
        ema_dict[symbol] = ema_df
    return ema_dict

def compute_macd(data_dict, fast=12, slow=26, signal=50):
    macd_dict = {}

    for symbol, df in data_dict.items():
        macd_df = pd.DataFrame(index=df.index)
        ema_fast = df['close'].ewm(span=fast, adjust=False).mean()
        ema_slow = df['close'].ewm(span=slow, adjust=False).mean()

        macd_df['macd'] = ema_fast - ema_slow
        macd_df['signal_line'] = macd_df['macd'].ewm(span=signal, adjust=False).mean()
        macd_df['histogram'] = macd_df['macd'] - macd_df['signal_line']

        macd_dict[symbol] = macd_df

    return macd_dict

def compute_bollinger_bands(data_dict, window=20, num_std=2):
    bb_dict = {}

    for symbol, df in data_dict.items():
        bb_df = pd.DataFrame(index=df.index)
        rolling_mean = df['close'].rolling(window=window).mean()
        rolling_std = df['close'].rolling(window=window).std()
        bb_df['bb_upper'] = rolling_mean + num_std * rolling_std
        bb_df['bb_lower'] = rolling_mean - num_std * rolling_std
        bb_dict[symbol] = bb_df

    return bb_dict

def compute_stochastic_oscillator(data_dict, k_window=14, d_window=3):
    stoch_dict = {}

    for symbol, df in data_dict.items():
        stoch_df = pd.DataFrame(index=df.index)
        lowest_low = df['low'].rolling(window=k_window).min()
        highest_high = df['high'].rolling(window=k_window).max()
        stoch_df['%K'] = ((df['close'] - lowest_low) / (highest_high - lowest_low)) * 100
        stoch_df['%D'] = stoch_df['%K'].rolling(window=d_window).mean()
        stoch_dict[symbol] = stoch_df

    return stoch_dict

def compute_obv(data_dict, signal_span=20):
    obv_dict = {}
    
    for symbol, df in data_dict.items():
        obv = [0]
        for i in range(1, len(df)):
            if df['close'].iloc[i] > df['close'].iloc[i - 1]:
                obv.append(obv[-1] + df['volume'].iloc[i])
            elif df['close'].iloc[i] < df['close'].iloc[i - 1]:
                obv.append(obv[-1] - df['volume'].iloc[i])
            else:
                obv.append(obv[-1])
        
        obv_series = pd.Series(obv, index=df.index)
        obv_signal = obv_series.ewm(span=signal_span).mean()
        obv_hist = obv_series - obv_signal

        obv_df = pd.DataFrame({
            'obv': obv_series,
            'obv_signal': obv_signal,
            'obv_histogram': obv_hist
        })
        obv_dict[symbol] = obv_df
        
    return obv_dict

def compute_atr(data_dict, window=14):
    atr_dict = {}

    for symbol, df in data_dict.items():
        high = df['high']
        low = df['low']
        close = df['close']
        prev_close = close.shift(1)

        tr = pd.concat([
            (high - low).abs(),
            (high - prev_close).abs(),
            (low - prev_close).abs()
        ], axis=1).max(axis=1)

        atr = tr.ewm(span=window, adjust=False).mean()  # Exponential average
        atr_df = pd.DataFrame({'atr': atr}, index=df.index)

        atr_dict[symbol] = atr_df

    return atr_dict

def compute_vwap(data_dict):
    vwap_dict = {}
    for symbol, df in data_dict.items():
        df = df.copy()
        typical_price = (df['high'] + df['low'] + df['close']) / 3
        cumulative_vp = (typical_price * df['volume']).cumsum()
        cumulative_vol = df['volume'].cumsum()
        df['vwap'] = cumulative_vp / cumulative_vol
        vwap_dict[symbol] = df[['vwap']]
    return vwap_dict

In [ ]:
# Feature Function Calls
returns = compute_returns(data)
volatility = compute_volatility(returns)
volumes = extract_volume(data)
rsi = compute_rsi(data)
sma = compute_sma(data)
ema = compute_ema(data)
macd = compute_macd(data)
bollinger_bands = compute_bollinger_bands(data)
stochastic = compute_stochastic_oscillator(data)
obv = compute_obv(data)
atr = compute_atr(data)
vwap = compute_vwap(data)

# Return CSVs
"""for symbol, df in returns.items():
    df.to_csv(f"returns_{symbol}_4h.csv")"""

"""for symbol, df in volatility.items():
    df.to_csv(f"volatility_{symbol}_4h.csv")"""

"""for symbol, df in rsi.items():
    df.to_csv(f"rsi_{symbol}_4h.csv")"""

"""for symbol, df in sma.items():
    df.to_csv(f"sma_{symbol}_4h.csv")"""
    
"""for symbol, df in ema.items():
    df.to_csv(f"ema_{symbol}_4h.csv")"""

"""for symbol, df in macd.items():
    df.to_csv(f"macd_{symbol}_4h.csv")"""

"""for symbol, df in bollinger_bands.items():
    df.to_csv(f"bollinger_{symbol}_4h.csv")"""

"""for symbol, df in stochastic.items():
    df.to_csv(f"stochastic_{symbol}_4h.csv")"""

"""for symbol, df in obv.items():
    df.to_csv(f"obv_{symbol}_4h.csv")"""

"""for symbol, df in atr.items():
    df.to_csv(f"atr_{symbol}_4h.csv")"""

"""for symbol, df in vwap.items():
    df.to_csv(f"vwap_{symbol}_4h.csv")"""

"""for symbol, df in volumes.items():
    df.to_csv(f"volume_{symbol}_4h.csv")"""

"""for symbol, df in roc.items():
    df.to_csv(f"roc_{symbol}_4h.csv")"""

'for symbol, df in roc.items():\n    df.to_csv(f"roc_{symbol}_4h.csv")'

In [18]:
# Step 3: Merge Features Tables

def merge_raw_and_features(symbols, feature_folders, base_feature_path, raw_data_path, output_path):
    os.makedirs(output_path, exist_ok=True)

    for symbol in symbols:
        # Load raw CSV
        raw_file = os.path.join(raw_data_path, f"raw_{symbol}USDT_4h.csv")
        df = pd.read_csv(raw_file, index_col=0, parse_dates=True)

        # Merge features
        for feature, folder in feature_folders.items():
            feature_file = os.path.join(base_feature_path, folder, f"{feature.lower()}_{symbol}USDT_4h.csv")
            if os.path.exists(feature_file):
                feature_df = pd.read_csv(feature_file, index_col=0, parse_dates=True)
                df = df.join(feature_df, how='left')  # join by timestamp index
            else:
                print(f"Missing file: {feature_file}")

        # Save merged CSV
        output_file = os.path.join(output_path, f"merged_{symbol}USDT_4h.csv")
        df.to_csv(output_file)
        print(f"Saved merged CSV: {output_file}")


In [21]:
# Merge Feature Function Call
symbols = ['BTC', 'ETH', 'XRP', 'BNB', 'SOL']

feature_folders = {
    "rsi": "RSI",
    "sma": "SMA",
    "ema": "EMA",
    "macd": "MACD",
    "bollinger": "Bollinger Bands",
    "stochastic": "Stochastic Oscillator",
    "obv": "OBV",
    "atr": "ATR",
    "vwap": "VWAP",
    "returns": "Returns",
    "volatility": "Volatility"
}

raw_data_path = r"C:\Users\yulig\Desktop\Class Project\Load Data\Step 1 Raw Data CSVs"

base_feature_path = r"C:\Users\yulig\Desktop\Class Project\Load Data\Step 2 Features CSVs"
output_path = os.path.join(base_feature_path, "Merged")

merge_raw_and_features(symbols, feature_folders, base_feature_path, raw_data_path, output_path)

Saved merged CSV: C:\Users\yulig\Desktop\Class Project\Load Data\Step 2 Features CSVs\Merged\merged_BTCUSDT_4h.csv
Saved merged CSV: C:\Users\yulig\Desktop\Class Project\Load Data\Step 2 Features CSVs\Merged\merged_ETHUSDT_4h.csv
Saved merged CSV: C:\Users\yulig\Desktop\Class Project\Load Data\Step 2 Features CSVs\Merged\merged_XRPUSDT_4h.csv
Saved merged CSV: C:\Users\yulig\Desktop\Class Project\Load Data\Step 2 Features CSVs\Merged\merged_BNBUSDT_4h.csv
Saved merged CSV: C:\Users\yulig\Desktop\Class Project\Load Data\Step 2 Features CSVs\Merged\merged_SOLUSDT_4h.csv


In [22]:
# Step 3: Merge Features Tables

def merge_raw_and_features(symbols, feature_folders, base_feature_path, raw_data_path, output_path):
    os.makedirs(output_path, exist_ok=True)

    for symbol in symbols:
        raw_file = os.path.join(raw_data_path, f"raw_{symbol}USDT_4h.csv")
        try:
            df = pd.read_csv(raw_file, index_col=0, parse_dates=True)
        except FileNotFoundError:
            print(f"❌ Missing raw file: {raw_file}")
            continue

        for feature, folder in feature_folders.items():
            feature_file = os.path.join(base_feature_path, folder, f"{feature.lower()}_{symbol}USDT_4h.csv")
            if os.path.exists(feature_file):
                feature_df = pd.read_csv(feature_file, index_col=0, parse_dates=True)
                df = df.join(feature_df, how='left')  # merge on timestamp
            else:
                print(f"⚠️ Missing feature file: {feature_file}")

        output_file = os.path.join(output_path, f"merged_{symbol}USDT_4h.csv")
        df.to_csv(output_file)
        print(f"✅ Saved merged CSV: {output_file}")
